In [177]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='your module name')

#### 加载自制中文数据集、分词、去除停用词

In [178]:
data = pd.read_csv('smalldata_washed.csv')
rumor = data['rumor'].to_list()
reverse = data['reverse'].to_list()
rumor_class = len(rumor)*[1]
reverse_class = len(reverse)*[0]
data = rumor + reverse
data_class = rumor_class + reverse_class
#后续添加数据要放在raw目录下，使用.xlsx格式将后续标注的数据加入data
raw_data_list = os.listdir('raw')
data_list = []
good_name = re.compile(r'^(?!(\~\$)).*(.xlsx)')
for i in raw_data_list:
    if good_name.match(i):
    #    data_list.append(i)
        temp = pd.read_excel('raw/'+i)
        temp.fillna('',inplace=True)
        temp_rumor = [x.strip() for x in  temp['谣言'].to_list()  if x.strip()!='']
        temp_reverse = [x.strip() for x in  temp['真相'].to_list()  if x.strip()!='']
        temp_rumor_class = len(temp_rumor)*[1]
        temp_reverse_class = len(temp_reverse)*[0]
        temp_data = temp_rumor + temp_reverse
        temp_data_class = temp_rumor_class + temp_reverse_class
        for i in range( len(temp_data)):
            temp_data[i]=re.sub(r'[\,\'\ ]|(\n)','',temp_data[i])
    data = data + temp_data
    data_class = data_class + temp_data_class
with open('data.txt','w') as f:
    for i in range(len(data)):
        f.write(data[i]+'\t'+str(data_class[i])+'\n')
        data_list =list( zip(data, data_class))
random.shuffle(data_list)
data, data_class = zip(*data_list)
index = ['rumor{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(data)), data_class)]
df = pd .DataFrame(data=zip(data_class,data), columns=["rumor","text"], index=index)

# 去除停用词
with open ('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# jieba分词
import jieba
for i in range(len(df)):
    words = jieba.cut(df["text"].iloc[i],cut_all=False)
    words = [word for word in words if word not in stopwords]
    df["text"].iloc[i] = ' '.join(words) 

/var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/ipykernel_1407/1622170239.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"].iloc[i] = ' '.join(words)


In [179]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=df.text).toarray()
len(tfidf.vocabulary_)

/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


4021

In [180]:
tfidf_docs = pd.DataFrame(tfidf_docs)
#通过减去每个文档（词袋向量）的平均值来中心化文档
tfidf_docs = tfidf_docs - tfidf_docs.mean()
tfidf_docs.shape
df.rumor.sum()

330

#### 基于PCA的短消息语义分析

In [181]:
from sklearn.decomposition import PCA
k=min(len(tfidf.vocabulary_),len(df))
k=100
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)


In [182]:
nums = pca.singular_values_.tolist()

In [183]:
'''
for i in range(len(nums)):
    if(sum(nums[:i+1])/base>0.5):
        k=i
        break
'''


'\nfor i in range(len(nums)):\n    if(sum(nums[:i+1])/base>0.5):\n        k=i\n        break\n'

In [184]:
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
pca_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,...,topic96,topic97,topic98,topic99
rumor0,-0.041,-0.038,0.021,-0.006,...,-0.035,0.061,-0.080,0.075
rumor1,0.006,0.038,-0.095,-0.013,...,0.089,-0.006,0.091,-0.007
rumor2,-0.033,-0.043,0.002,0.063,...,0.010,0.016,0.083,-0.019
rumor3,0.186,0.020,0.108,-0.114,...,-0.052,0.035,-0.021,0.001
rumor4!,0.219,0.279,-0.006,0.005,...,0.015,-0.041,-0.033,0.009
rumor5!,-0.054,-0.064,0.032,0.006,...,0.019,-0.043,-0.006,-0.059


In [185]:
tfidf.vocabulary_
#根据词项的频率对词汇表进行排序
#当对某个不按照最左边元素排序的序列解压并在排序后重新压缩时，可以使用zip(*sorted(zip(...)))
column_nums , terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))


In [186]:
weights = pd.DataFrame(pca.components_, columns=terms,
                       index = ['topic{}'.format(i) for i in range(pca.components_.shape[0])])
pd.options.display.max_columns = 8
weights.head(4).round(3)

,%,0,0t,1,...,鼓励,鼻出血,鼻翼,龋齿
topic0,-0.001,-0.001,-0.004,-0.003,...,-0.001,-0.005,-0.002,-0.003
topic1,-0.003,-0.001,-0.002,-0.011,...,-0.002,-0.008,-0.002,-0.004
topic2,-0.001,0.000,0.001,-0.009,...,-0.001,-0.002,0.000,-0.001
topic3,0.001,-0.000,0.003,0.008,...,-0.003,0.000,0.001,0.003


#### 基于截断的SVD的短消息语义分析（recommend）

In [187]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=k,n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)
svd_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,...,topic96,topic97,topic98,topic99
rumor0,-0.041,-0.039,0.021,-0.006,...,0.063,0.065,-0.040,0.012
rumor1,0.006,0.038,-0.095,-0.013,...,0.069,0.055,0.056,0.002
rumor2,-0.034,-0.043,0.002,0.063,...,-0.000,-0.004,0.048,0.008
rumor3,0.186,0.020,0.109,-0.114,...,0.024,0.027,-0.036,0.035
rumor4!,0.218,0.279,-0.005,0.006,...,0.001,0.004,-0.008,0.040
rumor5!,-0.054,-0.064,0.033,0.006,...,-0.020,-0.055,-0.055,-0.017


#### 基于LSA的垃圾短消息分类的效果(PCA/SVD+LDA)

In [188]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(svd_topic_vectors,df.rumor,test_size=0.5,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA(n_components=1)
lda = lda.fit(X_train_res, y_train_res)
df['LSA16_rumor'] = lda.predict(svd_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

过采样前训练集中类别0的数量： 237
过采样前训练集中类别1的数量： 162
过采样后训练集中类别0的数量： 237
过采样后训练集中类别1的数量： 237


0.722

In [189]:
import numpy as np
svd_topic_vectors = (svd_topic_vectors.T / np.linalg.norm(svd_topic_vectors,axis=1)).T
svd_topic_vectors.iloc[:10].dot(svd_topic_vectors.iloc[:10].T).round(1)

,rumor0,rumor1,rumor2,rumor3,...,rumor6!,rumor7,rumor8,rumor9
rumor0,1.0,-0.1,-0.1,0.1,...,-0.0,-0.0,0.0,-0.0
rumor1,-0.1,1.0,-0.1,-0.0,...,0.0,-0.1,0.1,-0.1
rumor2,-0.1,-0.1,1.0,0.1,...,-0.1,-0.0,-0.1,-0.1
rumor3,0.1,-0.0,0.1,1.0,...,0.1,0.0,-0.1,-0.0
rumor4!,-0.0,-0.1,-0.1,0.0,...,-0.0,-0.1,-0.1,0.2
rumor5!,-0.1,-0.0,0.1,-0.0,...,-0.0,-0.0,-0.0,-0.1
rumor6!,-0.0,0.0,-0.1,0.1,...,1.0,-0.0,0.1,-0.1
rumor7,-0.0,-0.1,-0.0,0.0,...,-0.0,1.0,0.0,0.2
rumor8,0.0,0.1,-0.1,-0.1,...,0.1,0.0,1.0,-0.1
rumor9,-0.0,-0.1,-0.1,-0.0,...,-0.1,0.2,-0.1,1.0


In [191]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lda, svd_topic_vectors, df.rumor, cv=5)
"Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2)

'Accuracy: 0.728 (+/- 0.057)'